In [ ]:
import numpy as np
import scanpy as sc
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sys
path_helper = ["C:\\","Users","vfriedrich","projects","monkey_IZI","git_documentation","scRNAseq_cross_species_primate_human","analysis","helper"]
sys.path.append(os.path.join(*path_helper))
import helperVDF as h
print(sys.executable)
from matplotlib.colors import Normalize

In [ ]:
pre = "MH95"
drive = 'F'
#species = 'cyno'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                                            pre = pre,
                                                                                            add_path = True)

read in from previous calculations

In [ ]:
_,base_table_path_MH93,_,_ = h.return_local_paths(drive = drive,pre = 'MH93',add_path = True)

In [ ]:
_,base_table_path_MH94,_,_ = h.return_local_paths(drive = drive,pre = 'MH94',add_path = True)

In [ ]:
_,base_table_path_H82,_,_ = h.return_local_paths(drive = drive,pre = "H82",add_path = False)
df_dge_H = pd.read_csv(os.path.join(base_table_path_H82,'H82_dge_tab_edgeR.csv'),index_col=0)
df_dge_H['FDR_0.25'] = df_dge_H['FDR'] <= 0.25
df_dge_H['FDR_0.5'] = df_dge_H['FDR'] <= 0.5
df_dge_H['FDR_0.2'] = df_dge_H['FDR'] <= 0.2

In [ ]:
df_dge_M_final = pd.read_csv(os.path.join(base_table_path_MH93,'MH93' + '_'+ 'df_dge_M_final.csv'),index_col=0)

In [ ]:
df_DGE_processed = pd.read_csv(os.path.join(base_table_path_MH93,'MH93' + '_df_DGE_processed.csv'),index_col=0)

In [ ]:
results_df_prop = pd.read_csv(os.path.join(base_table_path_MH94,'MH94' + '_results_df_prop_test.csv'),index_col=0)

In [ ]:
results_df_prop

In [ ]:
results_df_prop['ident'] = results_df_prop['celltype'] + results_df_prop['comp_M'] + results_df_prop['comp_H']

In [ ]:
df_DGE_processed['ident'] = df_DGE_processed['celltype'] + [item[-4:] for item in df_DGE_processed['comparison_vs_00hr_H']] + [item[-4:] for item in df_DGE_processed['comparison_vs_00hr_M']]

In [ ]:
df_DGE_processed_with_prop = pd.merge(df_DGE_processed, results_df_prop, on='ident', how='inner')

In [ ]:
df_DGE_processed_with_prop.rename(columns={'celltype_x':'celltype'},inplace=True)

In [ ]:
def map_significance(p_value):
    if p_value <= 0.0001:
        return '****'
    elif p_value <= 0.001:
        return '***'
    elif p_value <= 0.01:
        return '**'
    elif p_value <= 0.05:
        return '*'
    else:
        return ''

df_DGE_processed_with_prop['significance_star'] = df_DGE_processed_with_prop['p_value_prop_test'].apply(map_significance)

In [ ]:
df_DGE_processed_with_prop.to_csv(os.path.join(base_table_path,pre+ '_df_DGE_processed_with_prop.csv'))

In [ ]:
plt.figure(figsize=(14, 4))
# Set the aesthetic style of the plots
sns.set(style="whitegrid")

# Create a scatter plot with seaborn for categorical H and M
scatter = sns.scatterplot(
    x='ct+comp_00hr_H',
    y='comp_00hr_M',
    size='nr_FDR_both',
    hue='fraction_same_direction',
    data=df_DGE_processed_with_prop,
    palette='Purples',
    sizes=(100, 800),
    #alpha=1-df_DGE_processed_with_prop['p_value_prop_test'],
    edgecolor='black',
    legend=True
)

ax = plt.gca()
# Add color bar for 'fraction_same_direction'
#norm = plt.Normalize(0,1)
sm = plt.cm.ScalarMappable(cmap="Purples")
#sm.set_array([])
#cbar = plt.colorbar(sm, ax=ax)
#cbar.set_label('Fraction Same Direction')
#plt.colorbar(sm, label='Fraction Same Direction')
ax.set_ylim(-0.5, 1.5)
# Add labels and title

categories = sorted(df_DGE_processed_with_prop['comp_00hr_M'].unique())
category_to_position = {cat: i for i, cat in enumerate(categories)}

for i, row in df_DGE_processed_with_prop.iterrows():
    x_coord = row['ct+comp_00hr_H']
    y_coord = category_to_position[row['comp_00hr_M']]  # Map categorical value to numeric position
    
    # Retrieve the significance stars from the 'significance' column
    significance = row['significance_star']
    
    # Only annotate if there's a significance value (non-empty string)
    if significance:
        ax.text(
            x_coord, 
            y_coord + 0.18,  # Offset the numeric position for annotation
            significance, 
            ha='center', 
            va='bottom', 
            fontsize=14, 
            color='black'
        )


ax.set_ylim(-0.5, len(categories) - 0.5)

plt.xlabel('Human')
plt.ylabel('Cyno')
plt.title('DGE summary')
ax.legend(bbox_to_anchor=(1.05, 1))
plt.xticks(rotation=90)
plt.subplots_adjust(right=0.75)
plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_scatter_same_direction_nr_FDR_both.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.lines as mlines
import matplotlib.pyplot as plt

# Create a blank figure
plt.figure(figsize=(2, 2))
ax = plt.gca()

# Add a custom legend for significance stars
star_legend = [
    mlines.Line2D([], [], color='black', marker='', linestyle='none', label='**** $p \leq 0.0001$'),
    mlines.Line2D([], [], color='black', marker='', linestyle='none', label='***  $p \leq 0.001$'),
    mlines.Line2D([], [], color='black', marker='', linestyle='none', label='**   $p \leq 0.01$'),
    mlines.Line2D([], [], color='black', marker='', linestyle='none', label='*    $p \leq 0.05$'),
]

# Add the legend to the plot
ax.legend(handles=star_legend, loc='center', title='significance (prop.test)')

# Remove axes for clarity
ax.axis('off')

# Show the plot
plt.tight_layout()
plt.savefig(os.path.join(base_plots_path,pre + '_significance_legend.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)
h.print_main_versions()